In [1]:
import threading
import keyboard
import pyautogui
import pickle
from time import sleep, time

class Hook(threading.Thread):
    def __init__(self):
        super(Hook, self).__init__()
        self.daemon = True
        self.event = False
        self.record = False
        self.action_list = []

        keyboard.unhook_all()

    def run(self):
        print('Hooking Started')
        while True:
            key = keyboard.read_event(suppress=False)
            if key.name == 'f2':
                self.record = True
            elif key.name == 'f4':
                self.event = True
                break
def track_pos():
    action_time_list = []
    h = Hook()
    h.start()
    print('size:', pyautogui.size())
    print('기록을 시작합니다. 기록:F2, 정지:F4', pyautogui.size())
    last_time = time()
    
    while True:
        position = pyautogui.position()
        if h.event:
            print('\nTracking 종료')
            break
        if h.record:
            elapsed_time = time() - last_time
            action_time_list.append(((position.x, position.y), elapsed_time, h.action_list))
            h.action_list = []
            last_time = time()
            print(' <------ New Pos & Time & Action Recorded')
            h.record = False

        print(f'\r{position.x:4}, {position.y:4}', end='')
        sleep(0.01)

    h.join()
    keyboard.unhook_all()
    return action_time_list

def auto_click(action_time_list, repetitions=1):
    h = Hook()
    h.start()
    for _ in range(repetitions):
        for (x, y), elapsed, actions in action_time_list:
            pyautogui.moveTo(x, y)
            for action, value in actions:
                if action == 'mouse':
                    if value == 'left_click':
                        pyautogui.click()
                    elif value == 'right_click':
                        pyautogui.rightClick()
                elif action == 'key':
                    pyautogui.press(value)
            sleep(elapsed)
    h.join()
    keyboard.unhook_all()
    
def save_to_file(data, filename='actions.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load_from_file(filename='actions.pkl'):
    with open(filename, 'rb') as f:
        return pickle.load(f)
# 실행 코드 부분
if __name__ == "__main__":
    print("1. 마우스 및 키보드 액션 기록")
    print("2. 액션 재생")
    choice = input("선택하세요 (1/2): ")

    if choice == '1':
        print("마우스 위치, 클릭, 키보드 입력 및 시간을 추적합니다.")
        action_time_list = track_pos()
        print('\nPos, time & action list :', action_time_list)
        save_to_file(action_time_list)
        print("기록된 액션이 파일에 저장되었습니다.")
    elif choice == '2':
        action_time_list = load_from_file()
        repetitions = int(input("액션을 몇 번 반복하시겠습니까? "))
        for _ in range(repetitions):
            auto_click(action_time_list)


1. 마우스 및 키보드 액션 기록
2. 액션 재생
선택하세요 (1/2): 2
액션을 몇 번 반복하시겠습니까? 2
Hooking Started


KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    print("1. 마우스 및 키보드 액션 기록")
    print("2. 액션 재생")
    choice = input("선택하세요 (1/2): ")

    if choice == '1':
        print("마우스 위치, 클릭, 키보드 입력 및 시간을 추적합니다.")
        action_time_list = track_pos()
        print('\nPos, time & action list :', action_time_list)
        save_to_file(action_time_list)
        print("기록된 액션이 파일에 저장되었습니다.")
    elif choice == '2':
        action_time_list = load_from_file()
        repetitions = int(input("액션을 몇 번 반복하시겠습니까? "))
        for _ in range(repetitions):
            auto_click(action_time_list)
